In [ ]:
pip install tflite-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.1/580.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
pip install tensorflow

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from pathlib import Path
import gdown
import zipfile
import os

# Fungsi untuk mengunduh file dari Google Drive
def download_file_from_google_drive(file_id, dest_path):
    url = f"https://drive.google.com/uc?id=1_YY46WkTmBW0ySlq2ifZg2lHuIfI4OOP"
    gdown.download(url, dest_path, quiet=False)

# ID file Google Drive
file_id = 'YOUR_FILE_ID'  # Ganti dengan ID file Google Drive Anda

# Nama file untuk menyimpan dataset
dataset_zip = 'dataset.zip'

# Unduh dataset
download_file_from_google_drive(file_id, dataset_zip)

# Periksa apakah file yang diunduh adalah file ZIP yang valid
if zipfile.is_zipfile(dataset_zip):
    # Ekstrak dataset
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall('dataset')
    print("File extracted successfully")
    # Hapus file zip setelah ekstraksi
    os.remove(dataset_zip)
else:
    raise Exception("Downloaded file is not a valid ZIP file")

# Path ke direktori dataset yang diekstrak
dataset_dir = Path('dataset')

# Buat dataset dari direktori
train_dataset = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Debug: Print jumlah batch dan label untuk verifikasi
for images, labels in train_dataset.take(1):
    print(f"Training batch shape: {images.shape}")
    print(f"Training labels: {labels.numpy()}")

for images, labels in validation_dataset.take(1):
    print(f"Validation batch shape: {images.shape}")
    print(f"Validation labels: {labels.numpy()}")

# Normalisasi dataset
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
normalized_validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Definisikan model dengan MobileNetV2 sebagai basis
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # Freeze the base model

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(train_dataset.class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Debug: Print summary model
model.summary()

# Latih model
history = model.fit(
    normalized_train_dataset,
    validation_data=normalized_validation_dataset,
    epochs=10
)

# Evaluasi model
loss, accuracy = model.evaluate(normalized_validation_dataset)
print(f'Validation loss: {loss:.2f}')
print(f'Validation accuracy: {accuracy:.2f}')

# Simpan model sebagai model TensorFlow
export_dir = 'saved_model'
tf.saved_model.save(model, export_dir)

# Konversi model ke TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Simpan model TensorFlow Lite
with open('barcode_reader_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model TFLite saved as barcode_reader_model.tflite")


Downloading...
From (original): https://drive.google.com/uc?id=1_YY46WkTmBW0ySlq2ifZg2lHuIfI4OOP
From (redirected): https://drive.google.com/uc?id=1_YY46WkTmBW0ySlq2ifZg2lHuIfI4OOP&confirm=t&uuid=6c4fe38b-59e9-4032-a8cc-c860611be907
To: /content/dataset.zip
100%|██████████| 99.3M/99.3M [00:01<00:00, 93.0MB/s]


File extracted successfully
Found 60 files belonging to 1 classes.
Using 48 files for training.
Found 60 files belonging to 1 classes.
Using 12 files for validation.
Training batch shape: (32, 224, 224, 3)
Training labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Validation batch shape: (12, 224, 224, 3)
Validation labels: [0 0 0 0 0 0 0 0 0 0 0 0]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
        

Model TFLite saved as barcode_reader_model.tflite
